In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

In [2]:
slurm = pd.read_csv("/mnt/research/CMSE495-SS24-ICER/slurm_usage/DID_FINAL_SLURM_OCT_2023.csv",delimiter="|")


In [3]:
slurm = slurm.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
slurm.head()

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
0,31496544,user_679,group_121,2023-03-21T11:13:45,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_489,...,0,28,0,21000M,NaN,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",NaN
1,31497932,user_679,group_121,2023-03-21T11:31:18,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_489,...,0,28,0,21000M,NaN,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",NaN
2,31993628,user_105,group_114,2023-03-22T18:19:12,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_661,...,0,12,0,150G,NaN,1,1,None assigned,"billing=23347,cpu=12,gres/gpu=8,mem=150G,node=1",NaN
3,39087660,user_652,group_054,2023-04-04T13:09:10,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_557,...,0,640,0,20G,NaN,10,10,None assigned,"billing=3112,cpu=640,mem=20G,node=10",NaN
4,59062820,user_188,group_046,2023-05-08T09:58:20,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [4]:
slurm.columns

Index(['JobID', 'User', 'Group', 'Submit', 'Start', 'End', 'Elapsed', 'State',
       'Account', 'AssocID', 'Partition', 'Timelimit', 'UserCPU', 'SystemCPU',
       'TotalCPU', 'CPUTime', 'CPUTimeRAW', 'ReqCPUS', 'AllocCPUS', 'ReqMem',
       'MaxRSS', 'ReqNodes', 'NNodes', 'NodeList', 'ReqTRES', 'AllocTRES'],
      dtype='object')

In [5]:
slurm.shape

(6376308, 26)

In [6]:
# Apply SLURM cleaner Function to SLURM dataset

# Preprocessing Data

In [7]:
# # convert Submit and Start to datetime, and filter rows with 'Unknown' start

# # Filter out rows where 'State' is "Cancelled" or Unknown
slurm = slurm[slurm['State'] != 'Cancelled']
slurm = slurm[slurm['Start']!= 'Unknown']
slurm = slurm.dropna(subset=['Start'])
slurm = slurm.dropna(subset=['Timelimit'])


slurm['Submit'] = pd.to_datetime(slurm['Submit'])
slurm['Start'] = pd.to_datetime(slurm['Start'])

slurm.head(5)

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
4,59062820,user_188,group_046,2023-05-08 09:58:20,2024-01-01 00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
5,59062828,user_188,group_046,2023-05-08 09:58:20,2024-01-01 01:00:49,2024-01-01T01:00:55,00:00:06,COMPLETED,account_017,assocID_676,...,240,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
6,59062836,user_188,group_046,2023-05-08 09:58:20,2024-01-01 01:00:49,2024-01-01T01:00:55,00:00:06,COMPLETED,account_017,assocID_676,...,240,40,40,8G,NaN,1,1,skl-030,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
7,59062844,user_188,group_046,2023-05-08 09:58:21,2024-01-01 00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
8,59062852,user_188,group_046,2023-05-08 09:58:21,2024-01-01 00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [8]:
def AggSLURMDat(dat):
    '''
    Aggregates all submitted jobs together, removing all batch/extern 
    jobs and including said information into a single job. Excludes
    jobs that do not have a clear '.batch' and '.extern' files

    args:

        dat - the slurm dataset 
    
    returns:

        out_df - the aggregated version of the slurm dataset
    '''
    
    # Initialize a list to hold DataFrames to concatenate later.
    dfs_to_concat = []

    # Iterate over each unique JobID, but try to reduce the number of DataFrame operations.
    for job in dat["JobID"].unique():  # Directly iterate over unique JobIDs.
        
        # Select all rows with the current JobID once and reuse this subset.
        jdat = dat[dat["JobID"] == job]

        # Find CPUTimeRAW values that occur exactly twice.
        cpu_time_list = jdat["CPUTimeRAW"].value_counts()
        cpu_time_list = cpu_time_list[cpu_time_list == 2].index

        for cpu_time in cpu_time_list:
            # Select rows with the current CPUTimeRAW once and reuse.
            ajob = jdat[jdat["CPUTimeRAW"] == cpu_time]

            batch_job = ajob[ajob["User"] == "user_258"]
            ag_job = ajob[ajob["User"] != "user_258"]

            # Skip iteration if conditions are met.
            if len(ag_job["User"]) == 0 or len(ag_job["User"]) == 2:
                print("skipping iteration of ", ag_job["User"] )
                continue

            # Directly setting values might be more efficient than using .loc in some cases.
            ag_job["MaxRSS"] = batch_job["MaxRSS"].values[0] if not batch_job.empty else ag_job["MaxRSS"]

            # Add the prepared DataFrame to the list.
            dfs_to_concat.append(ag_job)

    # Concatenate all DataFrames at once outside the loop.
    out_df = pd.concat(dfs_to_concat, ignore_index=True)

    return out_df


In [ ]:
# Never stops :(
slurm_cleaned = AggSLURMDat(slurm)
slurm_cleaned

skipping iteration of  234       user_600
181949    user_600
Name: User, dtype: object
skipping iteration of  239       user_600
181953    user_600
Name: User, dtype: object
skipping iteration of  244       user_600
181957    user_600
Name: User, dtype: object
skipping iteration of  249       user_600
181961    user_600
Name: User, dtype: object
skipping iteration of  254       user_600
181965    user_600
Name: User, dtype: object
skipping iteration of  259       user_600
181969    user_600
Name: User, dtype: object
skipping iteration of  264       user_600
181973    user_600
Name: User, dtype: object
skipping iteration of  268       user_600
181978    user_600
Name: User, dtype: object
skipping iteration of  272       user_600
181983    user_600
Name: User, dtype: object
skipping iteration of  276       user_600
181988    user_600
Name: User, dtype: object
skipping iteration of  419       user_612
182161    user_612
Name: User, dtype: object
skipping iteration of  423    user_371
439 

skipping iteration of  2931344    user_606
3041309    user_606
Name: User, dtype: object
skipping iteration of  5998730    user_606
6023668    user_606
Name: User, dtype: object
skipping iteration of  2976768    user_606
3043636    user_606
Name: User, dtype: object
skipping iteration of  3077852    user_606
3236915    user_606
Name: User, dtype: object
skipping iteration of  5905857    user_606
6014612    user_606
Name: User, dtype: object
skipping iteration of  5947252    user_606
6015700    user_606
Name: User, dtype: object
skipping iteration of  5947259    user_606
6015709    user_606
Name: User, dtype: object
skipping iteration of  5947260    user_606
6015712    user_606
Name: User, dtype: object
skipping iteration of  5951211    user_606
6016123    user_606
Name: User, dtype: object
skipping iteration of  5951212    user_606
6016126    user_606
Name: User, dtype: object
skipping iteration of  2931671    user_606
3041314    user_606
Name: User, dtype: object
skipping iteration of

skipping iteration of  833644    user_606
924225    user_606
Name: User, dtype: object
skipping iteration of  829595    user_606
923788    user_606
Name: User, dtype: object
skipping iteration of  829594    user_606
923787    user_606
Name: User, dtype: object
skipping iteration of  829590    user_606
923786    user_606
Name: User, dtype: object
skipping iteration of  826036    user_606
923590    user_606
Name: User, dtype: object
skipping iteration of  825580    user_606
923565    user_606
Name: User, dtype: object
skipping iteration of  819754    user_606
923531    user_606
Name: User, dtype: object
skipping iteration of  819124    user_606
923530    user_606
Name: User, dtype: object
skipping iteration of  816575    user_606
923523    user_606
Name: User, dtype: object
skipping iteration of  816574    user_606
923522    user_606
Name: User, dtype: object
skipping iteration of  816573    user_606
923521    user_606
Name: User, dtype: object
skipping iteration of  816519    user_606
9

skipping iteration of  2103      user_306
183071    user_306
Name: User, dtype: object
skipping iteration of  2106      user_306
183074    user_306
Name: User, dtype: object
skipping iteration of  2109      user_306
183077    user_306
Name: User, dtype: object
skipping iteration of  2112      user_306
183080    user_306
Name: User, dtype: object
skipping iteration of  2115      user_306
183083    user_306
Name: User, dtype: object
skipping iteration of  2118      user_306
183086    user_306
Name: User, dtype: object
skipping iteration of  2121      user_306
183089    user_306
Name: User, dtype: object
skipping iteration of  2124      user_306
183092    user_306
Name: User, dtype: object
skipping iteration of  2127      user_306
183095    user_306
Name: User, dtype: object
skipping iteration of  2130      user_306
183098    user_306
Name: User, dtype: object
skipping iteration of  2133      user_306
183101    user_306
Name: User, dtype: object
skipping iteration of  2136      user_306
1

skipping iteration of  2613      user_426
183763    user_426
Name: User, dtype: object
skipping iteration of  2616      user_344
183766    user_344
Name: User, dtype: object
skipping iteration of  2619      user_401
183769    user_401
Name: User, dtype: object
skipping iteration of  2627      user_482
183773    user_482
Name: User, dtype: object
skipping iteration of  2631      user_426
183777    user_426
Name: User, dtype: object
skipping iteration of  2634      user_426
183780    user_426
Name: User, dtype: object
skipping iteration of  2637      user_590
183783    user_590
Name: User, dtype: object
skipping iteration of  2648      user_373
183794    user_373
Name: User, dtype: object
skipping iteration of  2667    user_325
2671    user_325
Name: User, dtype: object
skipping iteration of  696434    user_325
909467    user_325
Name: User, dtype: object
skipping iteration of  405831    user_325
696410    user_325
Name: User, dtype: object
skipping iteration of  183821    user_325
40580

...

skipping iteration of  400464    user_332
427673    user_332
Name: User, dtype: object
skipping iteration of  400468    user_495
427676    user_495
Name: User, dtype: object
skipping iteration of  400469    user_495
427679    user_495
Name: User, dtype: object
skipping iteration of  400470    user_332
427682    user_332
Name: User, dtype: object
skipping iteration of  400474    user_495
427685    user_495
Name: User, dtype: object
skipping iteration of  400475    user_495
427688    user_495
Name: User, dtype: object
skipping iteration of  400476    user_332
427691    user_332
Name: User, dtype: object
skipping iteration of  400480    user_495
427694    user_495
Name: User, dtype: object
skipping iteration of  400481    user_495
427697    user_495
Name: User, dtype: object
skipping iteration of  400482    user_332
427700    user_332
Name: User, dtype: object
skipping iteration of  400486    user_495
427703    user_495
Name: User, dtype: object
skipping iteration of  400487    user_495
4

skipping iteration of  400771    user_495
428122    user_495
Name: User, dtype: object
skipping iteration of  400772    user_495
428125    user_495
Name: User, dtype: object
skipping iteration of  400773    user_332
428128    user_332
Name: User, dtype: object
skipping iteration of  400774    user_332
428131    user_332
Name: User, dtype: object
skipping iteration of  400775    user_495
428134    user_495
Name: User, dtype: object
skipping iteration of  400776    user_495
428137    user_495
Name: User, dtype: object
skipping iteration of  400780    user_332
428140    user_332
Name: User, dtype: object
skipping iteration of  400781    user_495
428143    user_495
Name: User, dtype: object
skipping iteration of  400785    user_495
428146    user_495
Name: User, dtype: object
skipping iteration of  400786    user_332
428149    user_332
Name: User, dtype: object
skipping iteration of  400787    user_332
428152    user_332
Name: User, dtype: object
skipping iteration of  400791    user_495
4

skipping iteration of  401149    user_495
428612    user_495
Name: User, dtype: object
skipping iteration of  401153    user_495
428615    user_495
Name: User, dtype: object
skipping iteration of  401154    user_495
428618    user_495
Name: User, dtype: object
skipping iteration of  401158    user_495
428621    user_495
Name: User, dtype: object
skipping iteration of  401159    user_495
428624    user_495
Name: User, dtype: object
skipping iteration of  401160    user_436
428627    user_436
Name: User, dtype: object
skipping iteration of  401163    user_428
428630    user_428
Name: User, dtype: object
skipping iteration of  401170    user_495
428634    user_495
Name: User, dtype: object
skipping iteration of  401171    user_495
428637    user_495
Name: User, dtype: object
skipping iteration of  401175    user_495
428640    user_495
Name: User, dtype: object
skipping iteration of  401176    user_495
428643    user_495
Name: User, dtype: object
skipping iteration of  401180    user_495
4

skipping iteration of  401525    user_186
429033    user_186
Name: User, dtype: object
skipping iteration of  401529    user_495
429036    user_495
Name: User, dtype: object
skipping iteration of  401530    user_186
429039    user_186
Name: User, dtype: object
skipping iteration of  401531    user_047
429042    user_047
Name: User, dtype: object
skipping iteration of  401537    user_495
429045    user_495
Name: User, dtype: object
skipping iteration of  401538    user_186
429048    user_186
Name: User, dtype: object
skipping iteration of  401542    user_495
429051    user_495
Name: User, dtype: object
skipping iteration of  401543    user_186
429054    user_186
Name: User, dtype: object
skipping iteration of  401547    user_495
429057    user_495
Name: User, dtype: object
skipping iteration of  401548    user_186
429060    user_186
Name: User, dtype: object
skipping iteration of  401552    user_495
429063    user_495
Name: User, dtype: object
skipping iteration of  401553    user_186
4

skipping iteration of  403402    user_700
430765    user_700
Name: User, dtype: object
skipping iteration of  403470    user_466
430768    user_466
Name: User, dtype: object
skipping iteration of  403522    user_012
430771    user_012
Name: User, dtype: object
skipping iteration of  403543    user_466
430774    user_466
Name: User, dtype: object
skipping iteration of  403566    user_428
430777    user_428
Name: User, dtype: object
skipping iteration of  403592    user_466
430781    user_466
Name: User, dtype: object
skipping iteration of  403599    user_466
430784    user_466
Name: User, dtype: object
skipping iteration of  403648    user_466
430787    user_466
Name: User, dtype: object
skipping iteration of  403754    user_466
430793    user_466
Name: User, dtype: object
skipping iteration of  403761    user_466
430796    user_466
Name: User, dtype: object
skipping iteration of  403782    user_466
430799    user_466
Name: User, dtype: object
skipping iteration of  403796    user_466
4

In [ ]:
# observing different units in Memory
print(set(slurm['ReqMem']))
print(set(slurm['MaxRSS']))

 Feature Engineering After Mapping user258 to find underutilization in Memery

In [ ]:
# slurm = slurm.dropna(subset=['ReqMem'])     # Dropping nan ReqMem as they all belong to user_258

# # convert ReqMem to a uniform measurement (assuming 'M' for MB and 'G' for GB and 'K' for KB)
# def convert_memory(mem_str):
#     if mem_str.endswith('K'):
#         return float(mem_str[:-1]) # remove 'K' and convert to float
#     elif mem_str.endswith('M'):
#         return float(mem_str[:-1]) * 1000
#     elif mem_str.endswith('G'):
#         return float(mem_str[:-1]) * 1000000  # convert GB to KB
#     elif mem_str.endswith('T'):
#         return float(mem_str[:-1]) * 1000000000 # convert Tb to KB
    
    

# slurm['ReqMem_KB'] = slurm['ReqMem'].apply(convert_memory)

# Function to list users that underutilize CPU, Time, and Nodes 

In [120]:
# The higher the number is the more the user underutilizes

def FindUnterutilizerSLURM(data, time_threshold, cpu_threshold, nodes_threshold):
    """
    Identifies users who are requesting more resources than they use.

    :param data: DataFrame containing SLURM job records.
    :param thresholds: Dictionary with keys 'cpu' and 'mem' for utilization thresholds.
    :return: DataFrame of underutilizing users and their job details.
    """
    
    # Function to parse time data columns: TimeLimit and Elapsed
    def parse_time_string(time_str):
        """Convert a time string into a timedelta object."""
        days = 0
        if '-' in time_str:
            days, time_str = time_str.split('-')
            days = int(days)
        
        parts = time_str.split(':')
        hours, minutes, seconds = map(int, parts) if len(parts) == 3 else (int(parts[0]), int(parts[1]), 0)
        return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)

    # # Convert Timelimit and Elapsed to timedeltas
    slurm['Timelimit_td'] = data['Timelimit'].apply(parse_time_string)
    slurm['Elapsed_td'] = data['Elapsed'].apply(parse_time_string)

    
    # Underutilized columns calculation 
    data['UnderUtilizeCPU'] = data.ReqCPUS - data.AllocCPUS    
    data['UnderUtilizeNodes'] = data.ReqNodes - data.NNodes  
    data['UnderUtilizeTime'] = (data['Timelimit_td'] - data['Elapsed_td']) / pd.Timedelta(hours=1)   # Convert UnderUtilizeTime to hours for comparison
    
    # Filter based on the specific thresholds
    underutilized = data[(data['UnderUtilizeTime'] > time_threshold) | 
                         (data['UnderUtilizeCPU'] > cpu_threshold) | 
                         (data['UnderUtilizeNodes'] > nodes_threshold)]

    
    return underutilized[['User','JobID','Group','State','Account','UnderUtilizeCPU','UnderUtilizeCPU','UnderUtilizeTime']]


In [121]:
FindUnterutilizerSLURM(slurm,7,0,0)

,Timelimit_td,User,JobID,Group,State,Account,UnderUtilizeCPU,UnderUtilizeCPU,UnderUtilizeTime
4,2 days 00:00:00,user_188,59062820,group_046,COMPLETED,account_017,0,0,47.997500
5,2 days 00:00:00,user_188,59062828,group_046,COMPLETED,account_017,0,0,47.998333
6,2 days 00:00:00,user_188,59062836,group_046,COMPLETED,account_017,0,0,47.998333
7,2 days 00:00:00,user_188,59062844,group_046,COMPLETED,account_017,0,0,47.997500
8,2 days 00:00:00,user_188,59062852,group_046,COMPLETED,account_017,0,0,47.997500
...,...,...,...,...,...,...,...,...,...
6375757,7 days 00:00:00,user_683,99253928,group_096,PREEMPTED,account_002,0,0,159.340278
6375758,7 days 00:00:00,user_186,99253932,group_003,COMPLETED,account_017,0,0,167.994722
6375764,7 days 00:00:00,user_683,99253956,group_096,PREEMPTED,account_002,0,0,159.340556
6375779,4 days 04:00:00,user_060,99254020,group_154,COMPLETED,account_017,0,0,87.934167
